## Automate Event Planning

In [ ]:
# Warning Control
from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
from crewai import Agent, Task, Crew
from dotenv import load_dotenv
import os
import json
from pprint import pprint
from IPython.display import Markdown, display

In [ ]:
# --------- Load keys  ----------
load_dotenv()
#openai_api_key = os.getenv("OPENAI_API_KEY")
#serper_api_key = os.getenv("SERPER_API_KEY")

#os.environ["SERPER_API_KEY"]= serper_api_key
#os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4.1-mini'

#### CrewAI Tools

In [ ]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

# Initialize the tools
search_tool = SerperDevTool()   # To search the web (depending on Serper)
scrape_tool = ScrapeWebsiteTool()  # To extract the content of a web page (scraping)

#### Creating Agents

In [ ]:
# ------- Agent1: Venue Coordinator -------
venue_coordinator = Agent(
    role = "Venue Coordinator (Eco-Friendly)",
    goal = "Continuously research, evaluate, recommend, and book an appropriate venue "
           "that align with event themes, audience needs, and budget constraints.",
    backstory = (
        "With a keen sense of space and understanding of event logistics, "
        "you excel at finding and securing venues that reinforce the event’s identity "
        "while meeting size, budget, and environmental constraints."
        "you are trusted to balance comfort, accessibility, and environmental goals "
        "when advising on venue selection."
    ),
    tools=[search_tool, scrape_tool],
    verbose = True,
)

In [ ]:
# ------- Agent2: Logistics Manager for the event -------
logistics_manager = Agent(
    role = "Logistics Manager",
    goal = "Manage all logistics for the event "
           "including catering and equipmen",
    backstory = (
        "Organized and detail-oriented, you ensure every logistical aspect of the event "
        "from catering to equipment setup is flawlessly executed."
        "You place networking lounges effectively, and coordinate with venue/vendors "
        "to enable live product demos safely."
    ),
    verbose = True,
    tools=[search_tool, scrape_tool]
)

In [ ]:
# ------- Agent3: Event marketing and attendee communication ------
marketing_communications_agent = Agent(
    role = "Marketing and Communications Agent",
    goal = "Effectively market the event and "
           "communicate with participants",
    backstory = 
            "Creative and communicative, "
            "you craft compelling messages and "
            "engage with potential attendees "
            "to maximize event exposure and participation.",
    verbose = True,
    tools=[search_tool, scrape_tool]
)

In [ ]:
# ------ Agent4: Risk and compliance ------ 
risk_compliance = Agent(
    role="Risk and Compliance",
    goal="Ensures permits, insurance, safety plan, and supplier compliance, "
         "with event day checklist.",
    backstory="Specializes in minimizing surprises and ensuring legal compliance "
          "and operational safety at events.",
   tools=[search_tool, scrape_tool], 
    verbose=True
)

#### Creating Venue Pydantic Object
##### Pydantic model for structuring place outputs

In [ ]:
from pydantic import BaseModel

# ---- Pydantic Models (structured outputs) ----

class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str
    price_estimate: float
    pros: list[str]
    cons: list[str]
    url: str

#### Creating Tasks

In [ ]:
# Task1: Venue research and selection
venue_task = Task(
    description = (
        "Find a {venue_type} in {event_city} suitable for {event_topic}. "
        "The venue must accommodate around {expected_participants} participants "
        "and support the following preferences: {preferences}. "
        "Ensure cost considerations fit within a ${budget} budget."
    ),
    expected_output = (
        "All the details of a specifically chosen"
        "venue you found to accommodate the event."
    ),
    human_input= True, # Enable user interaction before running for extra input.
    output_json= VenueDetails,# Return the results in a specific structured format.
    output_file= "venue_details.json", # Outputs the venue details as a JSON file.
    agent = venue_coordinator,
)

In [ ]:
# Task2: Sustainable logistics plan
logistics_task = Task(
    description = (
        "Coordinate a sustainable logistics plan for {event_topic} with {expected_participants} "
        "participants on {tentative_date}, and cover the requirements "
        "including catering arrangements (with a focus on green and local options), "
        "equipment setup, attendee flow and accessibility, "
        "signage/wayfinding, and waste and recycling management."
    ),
    expected_output = (
        "A structured logistics package containing:\n"
        "- Confirmation of all logistics arrangements "
        "including catering and equipment setup.\n"
        "- A checklist of confirmed vendor arrangements with timelines, "
        "service level agreements, and sustainability compliance.\n"
        "- Clear confirmation that all logistics support the event theme and budget."
    ),
    human_input= True,
    async_execution= True,
    agent = logistics_manager,
)

In [ ]:
# Task3: Marketing and communications plan.
marketing_task = Task(
    description = (
        "Promote the {event_topic} aiming to engage at least " 
        "{expected_participants} potential attendees."
        "Highlight the event’s unique features such as {preferences} "
        "and its sustainability theme. Strategies should include social media, "
        "email campaigns, press engagement, and partnerships with local organizations."
    ),
    expected_output = (
        " A detailed report in Markdown format includes:\n"
        " - Marketing activities and attendee engagement.\n"
        "- Target audience and key messages.\n"
        "- Press release draft.\n"
        "- Notes on community partnerships and outreach channels."
    ),
    async_execution= True,
    output_file= "marketing_report.md",    # Outputs the report as a text file
    agent = marketing_communications_agent,
)

In [ ]:
# Task5: Risk and compliance checklist and safety plan
risk_task = Task(
    description=(
        "Define required permits and licenses, insurance requirements, emergency safety plan, "
        "and supplier compliance processes (e.g., food, audio-visual, security). "
        "Create an event day checklist appropriate to the audience size, venue, "
        "and sustainability criteria for the Green Energy Startup Expo event."
    ),
    expected_output=(
        "Markdown file containing: Requirements list (permits/insurance), brief safety "
        "and evacuation plan, Supplier compliance checklist, and day-of checklist."
    ),
    output_file = "risk_compliance_checklist.md",
    agent=risk_compliance,
)

#### Creating the Crew

In [ ]:
event_management_crew = Crew(
    agents = [venue_coordinator, 
              logistics_manager, 
              marketing_communications_agent,
              risk_compliance],
    tasks = [venue_task,logistics_task, marketing_task, risk_task],
    verbose = True
)

#### Running the Crew

In [ ]:
event_details = {
    'event_topic': "Green Energy Startup Expo",
    'event_city': "Austin, TX",
    'tentative_date': "2025-10-25",
    'expected_participants': 500,
    'budget': 60000,
    'preferences': (
                    "dedicated spaces for cleantech startup showcases,"
                    "live product demos,"
                    "networking lounges,"
                    "green catering"
                    ),  
    'audience': "cleantech founders, investors, utilities, policymakers, community advocates, and students",
    'venue_type': "Exhibition Center"
}

In [ ]:
result = event_management_crew.kickoff(inputs = event_details)

#### Display the generated venue_details.json file.

In [ ]:
with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

#### Display the generated marketing_report.md file.

In [ ]:
display(Markdown("./marketing_report.md"))

#### Display the generated risk_compliance_checklist.md file.

In [ ]:
display(Markdown("./risk_compliance_checklist.md"))